
# Travel Planning System with Multi-Agent Architecture

This notebook defines multiple specialized agents for travel planning:

1. **WeatherAnalysisAgent** – predicts best travel months using weather data.
2. **HotelRecommenderAgent** – finds hotels matching user preferences.
3. **ItineraryPlannerAgent** – generates a daily itinerary using GPT-2.
4. **SummaryAgent** – composes a summary email and estimates costs.


In [1]:
!pip install scikit-learn sentence-transformers transformers torch numpy


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:

from typing import List, Dict
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sentence_transformers import SentenceTransformer
from transformers import pipeline


/Users/gabriele/Documents/PythonProjects/Book_example/ch10_streamlit_p_swagent/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

class WeatherAnalysisAgent:
    def __init__(self):
        self.model = RandomForestRegressor(n_estimators=100)

    def train(self, historical_data: Dict):
        X = np.array([[d['month'], d['latitude'], d['longitude']] for d in historical_data])
        y = np.array([d['weather_score'] for d in historical_data])
        self.model.fit(X, y)

    def predict_best_time(self, location: Dict) -> Dict:
        predictions = []
        for month in range(1, 13):
            prediction = self.model.predict([[
                month,
                location['latitude'],
                location['longitude']
            ]]).item()
            predictions.append({'month': month, 'score': float(prediction)})

        return {
            'best_months': sorted(predictions, key=lambda x: x['score'], reverse=True)[:3],
            'location': location
        }


In [4]:

class HotelRecommenderAgent:
    def __init__(self):
        self.encoder = SentenceTransformer('all-MiniLM-L6-v2')
        self.hotels_db = []
        self.hotels_embeddings = None

    def add_hotels(self, hotels: List[Dict]):
        self.hotels_db = hotels
        descriptions = [h['description'] for h in hotels]
        self.hotels_embeddings = self.encoder.encode(descriptions)

    def find_hotels(self, preferences: str, top_k: int = 5) -> List[Dict]:
        pref_embedding = self.encoder.encode([preferences])
        similarities = np.dot(self.hotels_embeddings, pref_embedding.T).flatten()
        top_indices = similarities.argsort()[-top_k:][::-1]
        return [
            {**self.hotels_db[i], 'similarity_score': float(similarities[i])}
            for i in top_indices
        ]


In [5]:

class ItineraryPlannerAgent:
    def __init__(self):
        self.planner = pipeline(
            "text-generation",
            model="gpt2",
            max_length=500,
            truncation=True,
            pad_token_id=50256
        )

    def create_itinerary(self, destination_info: Dict, weather_info: Dict,
                         hotel_info: Dict, duration: int) -> Dict:
        prompt = self._create_prompt(destination_info, weather_info, hotel_info, duration)
        response = self.planner(prompt)[0]['generated_text']
        return {
            'itinerary': response,
            'duration': duration,
            'destination': destination_info['name']
        }

    def _create_prompt(self, destination_info: Dict, weather_info: Dict,
                       hotel_info: Dict, duration: int) -> str:
        return f"""Create a {duration}-day itinerary for {destination_info['name']}.
        Weather: {weather_info['best_months'][0]['month']} is the best month.
        Hotel: Staying at {hotel_info[0]['name']}.
        Attractions: {', '.join(destination_info['attractions'])}."""


In [6]:

class SummaryAgent:
    def __init__(self):
        self.llm = pipeline(
            "text-generation",
            model="gpt2",
            max_length=1000,
            truncation=True,
            pad_token_id=50256
        )

    def calculate_total_price(self, hotel_info: Dict, duration: int) -> float:
        hotel_cost = hotel_info[0]['price'] * duration
        daily_expenses = 100
        return hotel_cost + daily_expenses * duration

    def create_email(self, trip_data: Dict, client_name: str) -> Dict:
        total_price = self.calculate_total_price(
            trip_data['recommended_hotels'],
            trip_data['itinerary']['duration']
        )
        prompt = f"""
        Dear {client_name},

        Based on your preferences, I'm pleased to present your travel plan:

        Destination: {trip_data['itinerary']['destination']}
        Duration: {trip_data['itinerary']['duration']} days
        Best time to visit: Month {trip_data['weather_analysis']['best_months'][0]['month']}

        Recommended Hotel: {trip_data['recommended_hotels'][0]['name']}

        Itinerary Overview:
        {trip_data['itinerary']['itinerary']}

        Estimated Total Cost: ${total_price}
        """
        response = self.llm(prompt)[0]['generated_text']
        return {
            'email_content': response,
            'total_price': total_price
        }


In [7]:

class TravelPlanningSystem:
    def __init__(self):
        self.weather_agent = WeatherAnalysisAgent()
        self.hotel_agent = HotelRecommenderAgent()
        self.itinerary_agent = ItineraryPlannerAgent()
        self.summary_agent = SummaryAgent()

    def setup(self, weather_data, hotel_data):
        self.weather_agent.train(weather_data)
        self.hotel_agent.add_hotels(hotel_data)

    def plan_trip(self, destination, preferences, duration, client_name):
        weather = self.weather_agent.predict_best_time(destination)
        hotels = self.hotel_agent.find_hotels(preferences)
        itinerary = self.itinerary_agent.create_itinerary(destination, weather, hotels, duration)
        summary = self.summary_agent.create_email({
            'weather_analysis': weather,
            'recommended_hotels': hotels,
            'itinerary': itinerary
        }, client_name)
        return summary

# Sample usage
historical_weather_data = [{'month': m, 'latitude': 41.9, 'longitude': 12.5, 'weather_score': 0.5 + 0.05 * m} for m in range(1,13)]
hotels_database = [
    {'name': 'Grand Hotel', 'description': 'Luxury hotel in city center with spa and restaurant', 'price': 300},
    {'name': 'Boutique Resort', 'description': 'Boutique hotel with personalized service', 'price': 250}
]
destination = {'name': 'Rome', 'latitude': 41.9, 'longitude': 12.5, 'attractions': ['Colosseum', 'Vatican']}
preferences = "Luxury hotel with spa and restaurant"
system = TravelPlanningSystem()
system.setup(historical_weather_data, hotels_database)
result = system.plan_trip(destination, preferences, 3, "John Smith")
print(result['email_content'])
print("Estimated Total Cost: $", result['total_price'])


Device set to use mps:0
Device set to use mps:0



        Dear John Smith,

        Based on your preferences, I'm pleased to present your travel plan:

        Destination: Rome
        Duration: 3 days
        Best time to visit: Month 12

        Recommended Hotel: Grand Hotel

        Itinerary Overview:
        Create a 3-day itinerary for Rome.
        Weather: 12 is the best month.
        Hotel: Staying at Grand Hotel.
        Attractions: Colosseum, Vatican.
                               
Campus: The Church of St. Paul's at the site of St. James and St. Mary at the site of the New Sanctuary
                              There are many sites at the site of St. James and St. Mary, which were constructed by St. Paul (7:15-22); the site of Pius Felix's church (2:20-23), which is mentioned by C.S. Lewis' poem, "The Cave of St. Paul" (17:4). In 1601, a Roman monastery was built there from which, as mentioned previously, the Romans used a great wall to erect a church. One site by the name of Vigna Cui di Medicis may even date as f